In [1]:
import numpy as np
import pandas as pd
import mlflow
import warnings
import pickle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [28]:
# sns.set()
warnings.filterwarnings("ignore")

In [3]:
train_sales = pd.read_csv("../data/train.csv")

In [4]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,87.284560
2013-01-02,6233.030493
2013-01-03,5693.112108
2013-01-04,5954.218834
2013-01-05,5337.751570
...,...
2015-07-27,9602.952466
2015-07-28,8174.953363
2015-07-29,7623.284305


In [5]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array

In [6]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [7]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

In [8]:
DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [9]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

TensorShape([846, 1])

In [10]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [11]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [12]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [13]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [14]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [15]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [16]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [17]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

# Model Training

In [18]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533
Trainable params: 533
Non-trainable params: 0
_________________________________________________________________


In [29]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2022/05/27 14:02:46 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2022/05/27 14:02:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ab85061e95974a95871ed3e06e034cae', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/200
1/1 [==============================] - 0s 254ms/step - loss: 0.3233 - val_loss: 0.2573
Epoch 2/200
1/1 [==============================] - 0s 209ms/step - loss: 0.3222 - val_loss: 0.2559
Epoch 3/200
1/1 [==============================] - 0s 249ms/step - loss: 0.3212 - val_loss: 0.2545
Epoch 4/200
1/1 [==============================] - 0s 293ms/step - loss: 0.3200 - val_loss: 0.2530
Epoch 5/200
1/1 [==============================] - 0s 281ms/step - loss: 0.3189 - val_loss: 0.2516
Epoch 6/200
1/1 [==============================] - 0s 309ms/step - loss: 0.3178 - val_loss: 0.2501
Epoch 7/200
1/1 [==============================] - 0s 299ms/step - loss: 0.3166 - val_loss: 0.2486
Epoch 8/200
1/1 [==============================] - 0s 275ms/step - loss: 0.3154 - val_loss: 0.2471
Epoch 9/200
1/1 [==============================] - 0s 306ms/step - loss: 0.3142 - val_loss: 0.2456
Epoch 10/200
1/1 [==============================] - 0s 229ms/step - loss: 0.3130 - val_loss: 0.2440
Epoch 11/

1/1 [==============================] - 0s 264ms/step - loss: 0.2105 - val_loss: 0.1196
Epoch 84/200
1/1 [==============================] - 0s 221ms/step - loss: 0.2092 - val_loss: 0.1182
Epoch 85/200
1/1 [==============================] - 0s 218ms/step - loss: 0.2079 - val_loss: 0.1168
Epoch 86/200
1/1 [==============================] - 0s 250ms/step - loss: 0.2066 - val_loss: 0.1154
Epoch 87/200
1/1 [==============================] - 0s 237ms/step - loss: 0.2053 - val_loss: 0.1141
Epoch 88/200
1/1 [==============================] - 0s 282ms/step - loss: 0.2041 - val_loss: 0.1127
Epoch 89/200
1/1 [==============================] - 0s 230ms/step - loss: 0.2028 - val_loss: 0.1114
Epoch 90/200
1/1 [==============================] - 0s 237ms/step - loss: 0.2016 - val_loss: 0.1101
Epoch 91/200
1/1 [==============================] - 0s 226ms/step - loss: 0.2003 - val_loss: 0.1087
Epoch 92/200
1/1 [==============================] - 0s 248ms/step - loss: 0.1991 - val_loss: 0.1075
Epoch 93/200


1/1 [==============================] - 0s 235ms/step - loss: 0.1402 - val_loss: 0.0520
Epoch 165/200
1/1 [==============================] - 0s 224ms/step - loss: 0.1397 - val_loss: 0.0516
Epoch 166/200
1/1 [==============================] - 0s 225ms/step - loss: 0.1393 - val_loss: 0.0512
Epoch 167/200
1/1 [==============================] - 0s 228ms/step - loss: 0.1388 - val_loss: 0.0508
Epoch 168/200
1/1 [==============================] - 0s 229ms/step - loss: 0.1383 - val_loss: 0.0505
Epoch 169/200
1/1 [==============================] - 0s 220ms/step - loss: 0.1379 - val_loss: 0.0501
Epoch 170/200
1/1 [==============================] - 0s 243ms/step - loss: 0.1374 - val_loss: 0.0497
Epoch 171/200
1/1 [==============================] - 0s 228ms/step - loss: 0.1370 - val_loss: 0.0493
Epoch 172/200
1/1 [==============================] - 0s 242ms/step - loss: 0.1365 - val_loss: 0.0490
Epoch 173/200
1/1 [==============================] - 0s 252ms/step - loss: 0.1361 - val_loss: 0.0486
Epoc

2022/05/27 14:03:39 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: 'PrefetchDataset' object has no attribute '_batch_size'


In [21]:
from time import gmtime, strftime

In [27]:
time = gmtime()
model.save('../models/LSTM_sales{time}.pkl')

INFO:tensorflow:Assets written to: ../models/LSTM_sales{time}.pkl\assets


INFO:tensorflow:Assets written to: ../models/LSTM_sales{time}.pkl\assets
